In [13]:
using Pkg
Pkg.add("CUDNN_jll")

   Resolving package versions...
  No Changes to `C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\Project.toml`
  No Changes to `C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\Manifest.toml`


In [14]:
using DataFrames
using Transformers
using Transformers.TextEncoders
using Transformers.HuggingFace
using TrillionDollarWords

### Load data

In [15]:
n = 10
data = load_training_sentences()
texts = filter(:split => n -> n == "test", data)[1:n, :]

Row,sentence,year,label,seed,sentence_splitting,event_type,split
,String,Int64,String7,Int64,Bool,String31,String7
1,"remained well below their levels at the beginning of the year, and that weaker demand and earlier declines in oil prices had been holding down consumer price inflation.",1996,hawkish,5768,true,meeting minutes,test
2,"A few participants also noted that uncertainty about the extent of resource slack in the economy was considerable and that it was quite possible that the economy could soon be operating close to potential, particularly if labor force participation rates did not turn up much while employment continued to register gains.",1996,neutral,5768,true,meeting minutes,test
3,inflation was projected to pick up gradually in association with a partial reversal of the decline in energy prices this year.,1996,neutral,5768,true,meeting minutes,test
4,They noted that the realization of such a development could make it harder for the Committee to achieve 2 percent inflation over the longer run.,1996,neutral,5768,true,meeting minutes,test
5,"In the view of one member, however, aggregate final demand was so strong that, with economic activity and the associated demand for labor having expanded at an unsustainable pace for some time, one could be reasonably confident that inflation would most likely pick up in the absence of policy action.",1996,hawkish,5768,true,meeting minutes,test
6,"In the circumstances, most members endorsed a proposal to delete as no longer necessary the previous summary statement relating to the risks to growth and inflation taken together.",1996,neutral,5768,true,meeting minutes,test
7,"In the staff forecast prepared for this meeting, the economy was seen as likely to expand at a moderate pace, supported by accommodative monetary policy and financial conditions.",1996,dovish,5768,true,meeting minutes,test
8,"Housing starts and the demand for new homes had declined further, house prices in many parts of the country were falling faster than they had towards the end of 2007, and inventories of unsold homes remained quite elevated.",1996,dovish,5768,true,meeting minutes,test
9,Pressures on resources would rise as the anticipated upturn and possible above-trend growth brought the economy closer to full capacity utilization.,1996,hawkish,5768,true,meeting minutes,test


### Get attributions

In [ ]:
# using CondaPkg
# CondaPkg.add("pytorch")
# CondaPkg.add("transformers"; version="4.15.0")
# CondaPkg.add("transformers-interpret")
# CondaPkg.add("cuDNN")

In [1]:
using PythonCall

    CondaPkg Found dependencies: C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\CondaPkg.toml
    CondaPkg Found dependencies: C:\Users\drobi\.julia\packages\PythonCall\wXfah\CondaPkg.toml
    CondaPkg Dependencies already up to date


In [2]:
transformers_interpret = PythonCall.pyimport("transformers_interpret")
transformers = PythonCall.pyimport("transformers")

Python: <module 'transformers' from 'C:\\Users\\drobi\\Desktop\\uni\\master_thesis\\CounterfactualExplanations.jl\\dev\\notebooks\\.CondaPkg\\env\\lib\\site-packages\\transformers\\__init__.py'>

In [3]:
# load pre-trained classifier and corresponding tokenizer
model = transformers.RobertaForSequenceClassification.from_pretrained("model", local_files_only=true)
tokenizer = transformers.AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa")

Python: PreTrainedTokenizerFast(name_or_path='gtfintechlab/FOMC-RoBERTa', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [4]:
scorer = transformers_interpret.SequenceClassificationExplainer(model, tokenizer, attribution_type="lig")

Python: <transformers_interpret.explainers.sequence_classification.SequenceClassificationExplainer object at 0x0000026835C80DF0>

In [16]:
attribs = scorer(texts[1, :].sentence, index=0, internal_batch_size=1)

Python: [('<s>', 0.0), ('rem', -0.09617849663936606), ('ained', -0.29702715615956665), ('well', 0.25551209840018674), ('below', -0.4015987185377847), ('their', -0.013975036770146217), ('levels', -0.13794157805498133), ('at', -0.002103001077563767), ('the', 0.14926277592717438), ('beginning', 0.08228264530334284), ('of', 0.08006335674570937), ('the', 0.24284418735078794), ('year', 0.20003154332566433), (',', 0.13104142887853437), ('and', 0.17795123590947837), ('that', 0.177958452206897), ('weaker', 0.07827504560711215), ('demand', 0.06225654464144761), ('and', 0.20338402474663345), ('earlier', -0.2643532357406369), ('declines', 0.14990056209456956), ('in', -0.3258371366392156), ('oil', -0.06976564966318043), ('prices', 0.011989646035101107), ('had', -0.15355389676950998), ('been', -0.10492307025848874), ('holding', -0.07771222018418246), ('down', -0.024919108109520634), ('consumer', 0.09788758630638592), ('price', 0.011979387105458176), ('inflation', -0.09257150340664654), ('.', 0.15492

In [17]:
attributions = pyconvert(Array{Tuple{String, Float64}}, attribs)

34-element Vector{Tuple{String, Float64}}:
 ("<s>", 0.0)
 ("rem", -0.09617849663936606)
 ("ained", -0.29702715615956665)
 ("well", 0.25551209840018674)
 ("below", -0.4015987185377847)
 ("their", -0.013975036770146217)
 ("levels", -0.13794157805498133)
 ("at", -0.002103001077563767)
 ("the", 0.14926277592717438)
 ("beginning", 0.08228264530334284)
 ("of", 0.08006335674570937)
 ("the", 0.24284418735078794)
 ("year", 0.20003154332566433)
 ⋮
 ("oil", -0.06976564966318043)
 ("prices", 0.011989646035101107)
 ("had", -0.15355389676950998)
 ("been", -0.10492307025848874)
 ("holding", -0.07771222018418246)
 ("down", -0.024919108109520634)
 ("consumer", 0.09788758630638592)
 ("price", 0.011979387105458176)
 ("inflation", -0.09257150340664654)
 (".", 0.15492799445950947)
 ("", -0.3214473479860127)
 ("</s>", 0.0)

### Mask the word attributions

In [18]:
cls = TrillionDollarWords.load_model(; output_hidden_states=true)

┌ Warning: fuse_unk is unsupported, the tokenization result might be slightly different in some cases.
└ @ Transformers.HuggingFace C:\Users\drobi\.julia\packages\Transformers\lD5nW\src\huggingface\tokenizer\utils.jl:42
┌ Warning: match token `<mask>` require to match with space on either side but that is not implemented here, the tokenization result might be slightly different in some cases.
└ @ Transformers.HuggingFace C:\Users\drobi\.julia\packages\Transformers\lD5nW\src\huggingface\tokenizer\utils.jl:42


BaselineModel(GPT2TextEncoder(
├─ TextTokenizer(MatchTokenization(CodeNormalizer(BPETokenization(GPT2Tokenization, bpe = CachedBPE(BPE(50000 merges))), codemap = CodeMap{UInt8 => UInt16}(3 code-ranges)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 50265, unk = <unk>, unki = 4),
├─ codemap = CodeMap{UInt8 => UInt16}(3 code-ranges),
├─ startsym = <s>,
├─ endsym = </s>,
├─ padsym = <pad>,
├─ trunc = 256,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}(<s>:<type=1> Input:<type=1> </s>:<type=1> (</s>:<type=1> Input:<type=1> </s>:<type=1>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(256))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_or_pad(256, <pad>, tail, tail)(target.token)
  ╰─ target[token] := TextEncode

In [125]:
toks = decode(cls.tkr, encode(cls.tkr, texts[1, :].sentence).token)

256-element Vector{String}:
 "<s>"
 "rem"
 "ained"
 " well"
 " below"
 " their"
 " levels"
 " at"
 " the"
 " beginning"
 " of"
 " the"
 " year"
 ⋮
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"

In [20]:
word_attributions = []
for (i, (dec_tok, attrib)) in enumerate(zip(toks, attributions))
    if startswith(dec_tok, "<")
        continue
    elseif length(word_attributions) == 0 || startswith(dec_tok, " ")
        push!(word_attributions, ([i], [attrib[1]], [attrib[2]]))
    else 
        last_processed = last(word_attributions)
        push!(last_processed[1], i)
        push!(last_processed[2], attrib[1])
        push!(last_processed[3], attrib[2])
    end
end

In [106]:
word_attributions

28-element Vector{Any}:
 ([2, 3], ["rem", "ained"], [-0.09617849663936606, -0.29702715615956665])
 ([4], ["well"], [0.25551209840018674])
 ([5], ["below"], [-0.4015987185377847])
 ([6], ["their"], [-0.013975036770146217])
 ([7], ["levels"], [-0.13794157805498133])
 ([8], ["at"], [-0.002103001077563767])
 ([9], ["the"], [0.14926277592717438])
 ([10], ["beginning"], [0.08228264530334284])
 ([11], ["of"], [0.08006335674570937])
 ([12], ["the"], [0.24284418735078794])
 ([13, 14], ["year", ","], [0.20003154332566433, 0.13104142887853437])
 ([15], ["and"], [0.17795123590947837])
 ([16], ["that"], [0.177958452206897])
 ⋮
 ([20], ["earlier"], [-0.2643532357406369])
 ([21], ["declines"], [0.14990056209456956])
 ([22], ["in"], [-0.3258371366392156])
 ([23], ["oil"], [-0.06976564966318043])
 ([24], ["prices"], [0.011989646035101107])
 ([25], ["had"], [-0.15355389676950998])
 ([26], ["been"], [-0.10492307025848874])
 ([27], ["holding"], [-0.07771222018418246])
 ([28], ["down"], [-0.024919108109520

In [21]:
sorted = sort(word_attributions, by = x -> -maximum(x[3]))

28-element Vector{Any}:
 ([4], ["well"], [0.25551209840018674])
 ([12], ["the"], [0.24284418735078794])
 ([19], ["and"], [0.20338402474663345])
 ([13, 14], ["year", ","], [0.20003154332566433, 0.13104142887853437])
 ([16], ["that"], [0.177958452206897])
 ([15], ["and"], [0.17795123590947837])
 ([31, 32], ["inflation", "."], [-0.09257150340664654, 0.15492799445950947])
 ([21], ["declines"], [0.14990056209456956])
 ([9], ["the"], [0.14926277592717438])
 ([29], ["consumer"], [0.09788758630638592])
 ([10], ["beginning"], [0.08228264530334284])
 ([11], ["of"], [0.08006335674570937])
 ([17], ["weaker"], [0.07827504560711215])
 ⋮
 ([8], ["at"], [-0.002103001077563767])
 ([6], ["their"], [-0.013975036770146217])
 ([28], ["down"], [-0.024919108109520634])
 ([23], ["oil"], [-0.06976564966318043])
 ([27], ["holding"], [-0.07771222018418246])
 ([2, 3], ["rem", "ained"], [-0.09617849663936606, -0.29702715615956665])
 ([26], ["been"], [-0.10492307025848874])
 ([7], ["levels"], [-0.13794157805498133]

In [51]:
idx_to_mask = []
for row in first(sorted, 10)
    append!(idx_to_mask, row[1])
end

12-element Vector{Any}:
  4
 12
 19
 13
 14
 16
 15
 31
 32
 21
  9
 29

In [78]:
masked_text = Vector{Char}()
for (i, token) in enumerate(toks)
    if startswith(token, "<")
        continue
    elseif i in idx_to_mask
        append!(masked_text, " [MASK]")
    else
        append!(masked_text, token)
    end
end

masked_text = String(masked_text)

"remained [MASK] below their levels at [MASK] beginning of [MASK] [MASK] [MASK] [MASK] [MASK] weaker demand [MASK] earlier [MASK] in oil prices had been holding down [MASK] price [MASK] [MASK]"

In [79]:
function load_model(; kwrgs...)
    model_name = "karoldobiczek/relitc-FOMC-CMLM"
    tkr = Transformers.load_tokenizer(model_name)
    cfg = Transformers.HuggingFace.HGFConfig(Transformers.load_config(model_name); kwrgs...)
    mod = Transformers.load_model(model_name, "ForMaskedLM"; config = cfg)

    return tkr, mod, cfg
end
cmlm_tkr, cmlm_model = load_model()

(BertTextEncoder(
├─ TextTokenizer(MatchTokenization(WordPieceTokenization(bert_uncased_tokenizer, WordPiece(vocab_size = 30522, unk = [UNK], max_char = 100)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 30522, unk = [UNK], unki = 101),
├─ startsym = [CLS],
├─ endsym = [SEP],
├─ padsym = [PAD],
├─ trunc = 512,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}([CLS]:<type=1> Input[1]:<type=1> [SEP]:<type=1> (Input[2]:<type=2> [SEP]:<type=2>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(512))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_and_pad(512, [PAD], head, tail)(target.token)
  ╰─ target[token] := TextEncodeBase.nested2batch(target.token)
  ╰─ target[segment] := TextEncodeBase.trunc_and_pad(512, 1, hea

In [ ]:
mask_idx = 104

encoded_sentence = encode(cls.tkr, texts[1, :].sentence)
tokens = copy(encoded_sentence.token)
# for i in 1:size(tokens)[2]
#     if i - 1 in idx_to_mask
#         encoded_sentence.token[:, i, :] = 0
#         encoded_sentence.token
#     # print(argmax(enc.token[:, i, :]))
# end
# encoded_sentence
tokens[:, idx_to_mask, :] .= 0
tokens[mask_idx, idx_to_mask, :] .= 1
encoded_sentence.token = tokens

In [ ]:
decode(cmlm_tkr, tokens)

### Fill in masks

In [149]:
function merge_tokens(tokens, idx_to_mask=[])
    merged_text = Vector{Char}()
    for (i, token) in enumerate(tokens)
        if startswith(token, "<")
            continue
        elseif i in idx_to_mask
            append!(merged_text, " [MASK]")
        else
            append!(merged_text, token)
        end
    end
    
    return String(merged_text)
end

merge_tokens (generic function with 2 methods)

In [162]:
function left_to_right_filling(tokens, mask_positions, model, tokenizer)
    if length(mask_positions) == 0
        return merge_tokens(tokens)
    end

    masked_text = merge_tokens(tokens, mask_positions)
    # print(masked_text * "\n")
    
    mask_positions = sort(mask_positions)
    next_position = popfirst!(mask_positions)
    
    out = decode(cmlm_tkr, cmlm_model(encode(cmlm_tkr, masked_text)).logit)
    # print(out)

    next_token = out[next_position]
    if startswith(next_token, "##")
        next_token = chop(next_token, head=2, tail=0)
    else
        next_token = " " * next_token
    end
    
    tokens[next_position + 1] = next_token

    return left_to_right_filling(tokens, mask_positions, model, tokenizer)
end

left_to_right_filling (generic function with 1 method)

In [163]:
left_to_right_filling(copy(toks), idx_to_mask, cmlm_model, cmlm_tkr)

"remained well below their levels at the beginning of the and to to the in demand and earlier declines in oil prices had been holding down consumer price inflation . ."

In [122]:
out = cmlm_model(encode(cmlm_tkr, masked_text))

(hidden_state = Float32[-0.26276532 -0.304134 … -0.4412557 0.8395367; -0.3135784 -0.24896632 … -0.5622759 -0.06703592; … ; 0.21413484 0.26681167 … 0.3372178 -0.5381135; -0.25502548 -0.17415507 … -0.120725214 -0.17302077;;;], attention_mask = NeuralAttentionlib.LengthMask{1, Vector{Int32}}(Int32[32]), logit = Float32[-6.7711673 -7.257094 … -6.7281137 -13.482519; -6.676459 -6.8120203 … -6.3364005 -12.82685; … ; -6.089307 -6.884933 … -7.574874 -10.116859; -4.113231 -5.430876 … -3.934112 -11.168427;;;])

In [70]:
size(out.logit)

(30522, 11, 2)

In [47]:
enc = encode(cmlm_tkr, ["[UNK] [MASK]"])
for i in 1:size(enc.token)[2]
    print(argmax(enc.token[:, i, :]))
end

CartesianIndex(102, 1)CartesianIndex(101, 1)CartesianIndex(104, 1)CartesianIndex(103, 1)

In [81]:
out = decode(cmlm_tkr, out.logit)

32×1 Matrix{String}:
 "."
 "and"
 "well"
 "below"
 "their"
 "levels"
 "at"
 "the"
 "beginning"
 "of"
 "august"
 "quarter"
 "to"
 ⋮
 "in"
 "oil"
 "prices"
 "had"
 "been"
 "holding"
 "down"
 "the"
 "price"
 "stability"
 "."
 "."

In [82]:
function merge_truncated_words(tokens, in_word="##", between_word="", separators=[".", "de"])
    decoded = Vector{Char}()
    for token in tokens
        token = string(token)
        if token in separators
            continue
        else
            if startswith(token, in_word)
                if length(decoded) == 0
                    append!(decoded, token)
                else
                    last = pop!(decoded)
                    new_token = last * chop(token, head=2, tail=0)
                    append!(decoded, new_token)
                end
            else
                append!(decoded, " " * token)
            end
        end
    end
    return decoded
end

merge_truncated_words (generic function with 4 methods)

In [85]:
String(merge_truncated_words(out)[2:end])

"and well below their levels at the beginning of august quarter to to of strong demand and earlier increases in oil prices had been holding down the price stability"

In [56]:
idx = [1, 3, 4]
[1, 2, 3, 4, 5][idx]

3-element Vector{Int64}:
 1
 3
 4

In [63]:
x = rand(4, 4)

4×4 Matrix{Float64}:
 0.480747   0.166508  0.897285   0.770908
 0.401932   0.578816  0.878291   0.501171
 0.0753838  0.932655  0.0544905  0.132249
 0.301956   0.469797  0.485735   0.935682

In [65]:
x[1:2, [2, 3]] .= 0

2×2 view(::Matrix{Float64}, 1:2, [2, 3]) with eltype Float64:
 0.0  0.0
 0.0  0.0